In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from itertools import permutations


In [ ]:
import numpy as np
import nibabel as nib
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt

class GMM:
    def __init__(self, n_components, max_iter=100, tol=1e-4):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol
        
    def initialize_parameters(self, X):
        # Improved initialization: avoid zeros and use k-means-like spread
        n_samples = X.shape[0]
        nonzero_idx = np.where(X > 0)[0]
        if len(nonzero_idx) < self.n_components:
            raise ValueError("Too few non-zero values for initialization")
        random_idx = np.random.choice(nonzero_idx, self.n_components, replace=False)
        self.means = X[random_idx]
        self.covariances = np.array([np.cov(X.T) + np.eye(X.shape[1]) * 1e-6 
                                   for _ in range(self.n_components)])
        self.weights = np.ones(self.n_components) / self.n_components
        
    def e_step(self, X):
        n_samples = X.shape[0]
        responsibilities = np.zeros((n_samples, self.n_components))
        for k in range(self.n_components):
            gaussian = multivariate_normal(mean=self.means[k], cov=self.covariances[k])
            responsibilities[:, k] = self.weights[k] * gaussian.pdf(X)
        responsibilities /= responsibilities.sum(axis=1, keepdims=True) + 1e-10
        return responsibilities
    
    def m_step(self, X, responsibilities):
        n_samples = X.shape[0]
        Nk = responsibilities.sum(axis=0) + 1e-10
        self.weights = Nk / n_samples
        self.means = np.dot(responsibilities.T, X) / Nk[:, np.newaxis]
        for k in range(self.n_components):
            diff = X - self.means[k]
            self.covariances[k] = np.dot(responsibilities[:, k] * diff.T, diff) / Nk[k]
            self.covariances[k] += np.eye(X.shape[1]) * 1e-6
            # Reinitialize if variance is too small
            if self.covariances[k][0, 0] < 1e-6:
                nonzero_idx = np.where(X > 0)[0]
                if len(nonzero_idx) > 0:
                    self.means[k] = X[np.random.choice(nonzero_idx)]
                    self.covariances[k] = np.cov(X.T) + np.eye(X.shape[1]) * 1e-6
        
    def fit(self, X):
        self.initialize_parameters(X)
        for iteration in range(self.max_iter):
            old_responsibilities = self.e_step(X)
            self.m_step(X, old_responsibilities)
            new_responsibilities = self.e_step(X)
            if np.mean(np.abs(new_responsibilities - old_responsibilities)) < self.tol:
                print(f"Converged after {iteration + 1} iterations")
                break
        print("\nGMM Component Parameters:")
        for k in range(self.n_components):
            mean = self.means[k][0]
            variance = self.covariances[k][0, 0]
            print(f"Component {k}: Mean = {mean:.4f}, Variance = {variance:.4f}, Weight = {self.weights[k]:.4f}")
        return self
    
    def predict(self, X):
        responsibilities = self.e_step(X)
        return np.argmax(responsibilities, axis=1)

    def get_pdf(self, X_range):
        pdfs = []
        for k in range(self.n_components):
            gaussian = multivariate_normal(mean=self.means[k], cov=self.covariances[k])
            pdf = self.weights[k] * gaussian.pdf(X_range)
            pdfs.append(pdf)
        return np.array(pdfs)

In [ ]:
# Load Data
def load_nii(file_path):
    img = nib.load(file_path)
    data = img.get_fdata()
    return data.flatten()

mri_path = "sald_031764_img.nii"
gm_mask_path = "sald_031764_probmask_graymatter.nii"  # Replace with actual file name
wm_mask_path = "sald_031764_probmask_whitematter.nii"  # Replace with actual file name
csf_mask_path = "sald_031764_probmask_csf.nii"  # Replace with actual file name

mri_data = load_nii(mri_path)
gm_prob = load_nii(gm_mask_path)
wm_prob = load_nii(wm_mask_path)
csf_prob = load_nii(csf_mask_path)

# Preprocess: Remove zero-intensity voxels
nonzero_mask = mri_data > 0
X = mri_data[nonzero_mask].reshape(-1, 1)
true_probs = np.vstack([gm_prob[nonzero_mask], wm_prob[nonzero_mask], csf_prob[nonzero_mask]]).T
true_labels = np.argmax(true_probs, axis=1)

# Fit GMM
n_components = 3
gmm = GMM(n_components=n_components,max_iter=20)
gmm.fit(X)
labels = gmm.predict(X)

# Map clusters to tissue types
cluster_to_tissue = {}
for cluster in range(n_components):
    cluster_mask = (labels == cluster)
    if cluster_mask.sum() > 0:
        cluster_probs = true_probs[cluster_mask].mean(axis=0)
        tissue_idx = np.argmax(cluster_probs)
        cluster_to_tissue[cluster] = tissue_idx
    else:
        print(f"Warning: Cluster {cluster} is empty. Assigning to CSF (2).")
        cluster_to_tissue[cluster] = 2

mapped_labels = np.array([cluster_to_tissue[label] for label in labels])

# Save segmented image
mri_img = nib.load(mri_path)
segmented_full = np.zeros_like(mri_data, dtype=np.uint8)
segmented_full[nonzero_mask] = mapped_labels
segmented = segmented_full.reshape(mri_img.shape).astype(np.uint8)
seg_img = nib.Nifti1Image(segmented, mri_img.affine)
nib.save(seg_img, "segmented_output.nii")
print("Segmented image saved as 'segmented_output.nii'")

In [ ]:
# Compute pointwise accuracy for different permutations
mask_labels = [0, 1, 2]
all_permutations = permutations(mask_labels)

best_accuracy = 0
best_perm = None
for perm in all_permutations:
    permuted_labels = np.array([perm[label] for label in labels])
    accuracy = np.mean(permuted_labels == true_labels)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_perm = perm

print(f"Best Accuracy: {best_accuracy:.4f} with permutation {best_perm}")

In [ ]:
tissue_names = ["Gray Matter", "White Matter", "CSF"]
colors = ["blue", "green", "red"]

# Plot histograms for each tissue type
plt.figure(figsize=(10, 6))
for tissue_idx in range(3):
    tissue_intensities = X[mapped_labels == tissue_idx]  # Get intensities for the tissue
    plt.hist(tissue_intensities, bins=100, alpha=0.6, color=colors[tissue_idx], label=tissue_names[tissue_idx])

plt.xlabel("Intensity")
plt.ylabel("Frequency")
plt.title("Frequency of Points vs. Intensity for Different Tissue Types")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Define the tissue names for reference
tissue_names = ["Gray Matter (GM)", "White Matter (WM)", "Cerebrospinal Fluid (CSF)"]

# Map GMM labels using the best permutation
label_mapping = {i: tissue_names[best_perm[i]] for i in range(n_components)}

# Extract GMM parameters
components = [
    {'mean': gmm.means[k][0], 'variance': gmm.covariances[k][0, 0], 'weight': gmm.weights[k], 'label': label_mapping[k]}
    for k in range(gmm.n_components)
]

# Generate x values for visualization
x = np.linspace(X.min(), X.max(), 1000)

# Plot each Gaussian component
plt.figure(figsize=(8, 5))
for component in components:
    mean = component['mean']
    std_dev = np.sqrt(component['variance'])
    weight = component['weight']
    label = component['label']
    
    pdf = weight * norm.pdf(x, mean, std_dev)
    plt.plot(x, pdf, label=f"{label}: Mean={mean:.2f}, Variance={component['variance']:.2f}, Weight={weight:.4f}")

plt.title('Gaussian Mixture Model Components with Tissue Labels')
plt.xlabel('Intensity')
plt.ylabel('Probability Density')
plt.legend()
plt.grid()
plt.show()


# **Analysis of Misclassification in GMM-based MRI Segmentation**

## **Where is the highest misclassification?**
The highest misclassification occurs at the **boundary regions between tissue types**, specifically:

### **1. Gray Matter (GM) & White Matter (WM) Boundary (~90 Intensity Range)**
- In the **GMM plot**, Gray Matter (GM) peaks at **83.02**, while White Matter (WM) peaks at **101.09**.
- However, in the **frequency plot**, the **blue (GM) and green (WM) distributions overlap around 85–100**.
- This overlap causes **misclassification**, where some GM voxels might be incorrectly labeled as WM and vice versa.

### **2. Cerebrospinal Fluid (CSF) & Gray Matter (GM) Boundary (~50–60 Intensity Range)**
- CSF peaks at **43.93** but has **a very high variance (~775.63)**, meaning it spreads widely.
- Some **higher-intensity CSF voxels (~50–60) overlap with the lower-intensity GM voxels**.
- This leads to **CSF being misclassified as GM** in some cases.

---

## **Why is this happening?**

### **1. Overlapping Intensity Distributions**
- The intensity values of GM, WM, and CSF are **not strictly separated**.
- Some GM voxels **share intensities with both CSF and WM**, making classification difficult.

### **2. High Variance in CSF**
- The **large variance (775.63) for CSF** suggests that its intensities are widely spread out.
- This means **some CSF voxels have intensities similar to GM**, causing **misclassification**.

### **3. Gaussian Assumption in GMM**
- The **GMM assumes that each tissue type follows a normal distribution**, but in reality, MRI intensities are **more complex**.
- If the real distributions are **skewed or multi-modal**, the **GMM clusters will not perfectly align with tissue boundaries**.

---

## **Summary of Misclassification Regions**
| Misclassification | Region (Intensity) | Cause |
|------------------|-----------------|------|
| **GM ↔ WM** | ~85–100 | Overlapping distributions |
| **CSF ↔ GM** | ~50–60 | High variance in CSF |
| **Possible WM mislabeling** | >110 | GMM misfitting |

